# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [32]:
import pandas as pd
import matplotlib.pyplot as plt


# Notebook Presentation

In [51]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [34]:
data = pd.read_csv('cost_revenue_dirty.csv')

# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [35]:
data.shape

(5391, 6)

In [36]:
data.isna().values.any()

False

In [37]:
data_dedupe = data.drop_duplicates()
data_dedupe.shape == data.shape

True

In [38]:
for col in data.columns:
    first_entry = data[col][0]
    print(type(first_entry))

<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`.

Note that *domestic* in this context refers to the United States.

In [40]:
for col in ['USD_Production_Budget', 'USD_Worldwide_Gross', 'USD_Domestic_Gross']:
    data[col] = data[col].astype(str).str.replace(',', '')
    data[col] = data[col].astype(str).str.replace('$', '')
    data[col] = pd.to_numeric(data[col])

/Users/mymac/PycharmProjects/100DaysOfCode/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [42]:
data['Release_Date'] = pd.to_datetime(data['Release_Date'])

### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [58]:
# What is the average production budget of the films in the data set?

mean_budget = data['USD_Production_Budget'].mean()
'$' + '{:,.2f}'.format(mean_budget)

'$31,113,737.58'

In [59]:
# What is the average worldwide gross revenue of films?
mean_worldwide_revenue = data['USD_Worldwide_Gross'].mean()
'$' + '{:,.2f}'.format(mean_worldwide_revenue)

'$88,855,421.96'

In [60]:
# What were the minimums for worldwide and domestic revenue?
min_worldwide_revenue = data['USD_Worldwide_Gross'].min()
print('$', '{:,.2f}'.format(min_worldwide_revenue))

min_domestic_revenue = data['USD_Domestic_Gross'].min()
print('$', '{:,.2f}'.format(min_domestic_revenue))

$ 0.00
$ 0.00


In [68]:
# Are the bottom 25% of films actually profitable or do they lose money?

data['Profit'] = data['USD_Worldwide_Gross'] - data['USD_Production_Budget']

profitability = data.sort_values('Profit').reset_index(drop=True)
print('The 25th percentile movie is not profitable')

quarter_percentile = len(data)//4
profitability.loc[quarter_percentile]

The 25th percentile movie is not profitable


Rank                                                                  4671
Release_Date                                           2011-04-22 00:00:00
Movie_Title              POM Wonderful Presents: The Greatest Movie Eve...
USD_Production_Budget                                              1800000
USD_Worldwide_Gross                                                 783733
USD_Domestic_Gross                                                  638476
Profit                                                             -377791
Name: 1347, dtype: object

In [69]:
# What are the highest production budget and highest worldwide gross revenue of any film?
max_budget = data['USD_Production_Budget'].max()
print('$', '{:,.2f}'.format(max_budget))

max_worldwide_revenue = data['USD_Worldwide_Gross'].max()
print('$', '{:,.2f}'.format(max_worldwide_revenue))

$ 425,000,000.00
$ 2,783,918,982.00


In [78]:
# How much revenue did the lowest and highest budget films make?
max_budget = data['USD_Production_Budget'].max()
max_budget_idx = data['USD_Production_Budget'].idxmax()
max_budget_name = data['Movie_Title'][max_budget_idx]
rev_of_max_budget = data['USD_Worldwide_Gross'][max_budget_idx]
print(f'{max_budget_name} cost ${"{:,.2f}".format(max_budget)} and made ${"{:,.2f}".format(rev_of_max_budget)}')

min_budget = data['USD_Production_Budget'].min()
min_budget_idx = data['USD_Production_Budget'].idxmin()
min_budget_name = data['Movie_Title'][min_budget_idx]
rev_of_min_budget = data['USD_Worldwide_Gross'][min_budget_idx]
print(f'{min_budget_name} cost ${"{:,.2f}".format(min_budget)} and made ${"{:,.2f}".format(rev_of_min_budget)}')

Avatar cost $425,000,000.00 and made $2,783,918,982.00
My Date With Drew cost $1,100.00 and made $181,041.00


# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

### Filtering on Multiple Conditions

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [41]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 